In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
import pandas as pd
from sklearn.feature_selection import SelectFromModel

In [ ]:
# training,test,devel data index extraction from partition file
partition  = pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/metadata/partition.csv")
partition_train_index=partition[(partition["Partition"]=="train")]["Id"].to_numpy()
partition_devel_index=partition[(partition["Partition"]=="devel")]["Id"].to_numpy()
partition_test_index=partition[(partition["Partition"]=="test")]["Id"].to_numpy()
partition_index = partition[(partition["Partition"]=="train") | (partition["Partition"]=="devel")|(partition["Partition"]=="test")]["Id"].to_numpy()

## FAU extraction Procedure using pyfeat

In [ ]:
from feat import Detector
import feat
from feat.utils.io import read_feat


detector = Detector(
    face_model="retinaface",
    landmark_model="mobilefacenet",
    au_model='xgb',
    emotion_model="resmasknet",
    facepose_model="img2pose",
)

from feat.utils.io import get_test_data_path
from feat.plotting import imshow
import os

# Helper to point to the test data folder
test_data_dir = get_test_data_path()

# Get the full path
single_face_img_path = os.path.join("./c3_muse_stress_2022/raw_data/faces/2/", "11000.jpg")

# Plot it
single_face_prediction = detector.detect_image(single_face_img_path)

single_face_prediction.facebox
single_face_prediction.aus
single_face_prediction.emotions
single_face_prediction.facepose # (in degrees)
single_face_prediction.to_csv("output3.csv", index=False)

# input_prediction = read_feat("output3.csv")
# figs = single_face_prediction.plot_detections(poses=True)

# plt.savefig("pitch.png")


In [ ]:
test_image = os.path.join("/Thesis_data/c3_muse_stress_2022/raw_data/faces/2/0.jpg")

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
f, ax = plt.subplots()
im = Image.open(single_face_img_path)
ax.imshow(im);
old_size=im.size
new_size = (200, 200)
new_im = Image.new("RGB", new_size)   ## luckily, this is already black!
box = tuple((n - o) // 2 for n, o in zip(new_size, old_size))
new_im.paste(im, box)

new_im.show()

In [ ]:
from PIL import Image, ImageOps
img = Image.open(test_image)
img_with_border = ImageOps.expand(img,border=25,fill='white')
img_with_border.save('imaged-with-border2.png')

In [ ]:
image_prediction = detector.detect_image(test_image)
# Show results
image_prediction

In [ ]:
figs = single_face_prediction.plot_detections(poses=False)

## Spectrogram extraction using parselmouth

In [ ]:
import parselmouth

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set() # Use seaborn's default style to make attractive graphs
plt.rcParams['figure.dpi'] = 100 # Show nicely large images in this notebook

snd = parselmouth.Sound("audio/the_north_wind_and_the_sun.wav")
plt.figure()
plt.plot(snd.xs(), snd.values.T)
plt.xlim([snd.xmin, snd.xmax])
plt.xlabel("time [s]")
plt.ylabel("amplitude")
# plt.show() # or 
plt.savefig("sound.png")

def draw_spectrogram(spectrogram, dynamic_range=70):
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    sg_db = 10 * np.log10(spectrogram.values)
    plt.pcolormesh(X, Y, sg_db, vmin=sg_db.max() - dynamic_range, cmap='afmhot')
    plt.ylim([spectrogram.ymin, spectrogram.ymax])
    plt.xlabel("time [s]")
    plt.ylabel("frequency [Hz]")

def draw_intensity(intensity):
    plt.plot(intensity.xs(), intensity.values.T, linewidth=3, color='w')
    plt.plot(intensity.xs(), intensity.values.T, linewidth=1)
    plt.grid(False)
    plt.ylim(0)
    plt.ylabel("intensity [dB]")

intensity = snd.to_intensity()
spectrogram = snd.to_spectrogram()
plt.figure()
draw_spectrogram(spectrogram)
plt.twinx()
draw_intensity(intensity)
plt.xlim([snd.xmin, snd.xmax])

plt.savefig("sound1.png")

def draw_pitch(pitch):
    # Extract selected pitch contour, and
    # replace unvoiced samples by NaN to not plot
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values==0] = np.nan
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=5, color='w')
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=2)
    plt.grid(False)
    plt.ylim(0, pitch.ceiling)
    plt.ylabel("fundamental frequency [Hz]")

pitch = snd.to_pitch()
# If desired, pre-emphasize the sound fragment before calculating the spectrogram
pre_emphasized_snd = snd.copy()
pre_emphasized_snd.pre_emphasize()
spectrogram = pre_emphasized_snd.to_spectrogram(window_length=0.03, maximum_frequency=8000)
plt.figure()
draw_spectrogram(spectrogram)
plt.twinx()
draw_pitch(pitch)
plt.xlim([snd.xmin, snd.xmax])
plt.savefig("pitch.png")
# plt.show()